In [17]:
import pandas as pd
combined_dataset = pd.read_pickle('../combined_dataset_notes.pkl')

In [18]:

combined_dataset = combined_dataset.drop(['HADM_ID'], axis=1)
print(combined_dataset.shape)
combined_dataset.head()

(45526, 504)


,text,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
47366,"[chest, pa, lat, clip, clip, number, radiology...",7,0,35,462.0,0.0,24.0,26.0,1.0,104.0,...,0,1,0,0,0,0,0,0,0,1
47024,"[pm, liver, gallbladder, u, single, organ, cli...",5,1,60,462.0,-6.0,24.0,17.0,1.0,104.0,...,0,1,0,0,0,1,0,0,0,0
9046,"[chest, portable, ap, clip, clip, number, radi...",13,0,49,462.0,0.0,24.0,26.0,1.0,104.0,...,0,0,0,0,0,1,0,0,0,1
843,"[pm, chest, pa, lat, clip, clip, number, radio...",5,1,61,462.0,2.0,24.0,28.0,1.0,103.0,...,0,0,0,0,0,0,0,1,0,1
43466,"[pm, chest, portable, ap, different, physician...",5,0,54,462.0,0.0,23.0,25.0,1.0,106.0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
only_text = combined_dataset[["text","G1","G2","G3","G4","G5","G6","G7","G8","G9","G10","G11","G12","G13","G14","G15","G16","G17","G18","G19","G20"]]
only_text.shape

(45526, 21)

In [20]:
!pip install --upgrade gensim
!pip install -U gensim

Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)
Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)


In [6]:

from gensim.models import Word2Vec

In [8]:
#pass text here..
#model = Word2Vec(combined_dataset.text,size=750, window=10, min_count=5, workers=56)#play around with param
# summarize the loaded model
#print(model)
# summarize vocabulary
#words = list(model.wv.vocab)
#print(words)
# access vector for one word
#print(model['sentence'])
# save model
#model.save('model750-10-5.bin')
# load model
new_model = Word2Vec.load('model750-10-5.bin')
print(new_model)

Word2Vec(vocab=23960, size=750, alpha=0.025)


In [75]:
y = only_text.iloc[:,1:]
x = only_text.iloc[:,:1]

In [76]:
x.shape

(45526, 1)

In [77]:
import numpy as np
WordVectorz=dict(zip(new_model.wv.index2word,new_model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =750 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
AverageEmbeddingVectorizer(WordVectorz)
#AverageEmbeddingVectorizer(WordVectorz).fit(x.text,y)
#x.text = AverageEmbeddingVectorizer(WordVectorz).transform(x.text)
#x_test = AverageEmbeddingVectorizer(WordVectorz).transform(x_test)
#x_val = AverageEmbeddingVectorizer(WordVectorz).transform(x_val)

In [78]:
from sklearn.model_selection import train_test_split
seed=243 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=seed)
#x_train,x_val,y_train,y_val = train_test_split(x_tra,y_tra,test_size=0.25,random_state=seed)
print(x_train.shape,x_val.shape,y_train.shape,y_test.shape)

(38697, 1) (9675, 1) (38697, 20) (6829, 20)


In [38]:
x_train_df = pd.DataFrame(x_train)
x_train_df.columns

Index(['text'], dtype='object')

In [26]:
x_train_df#can we manually do avg row wise n put?
#as in?
# x_train = np.mean(x_train)

,text
30409,-0.068167
50215,-0.410700
7356,-0.071999
11076,0.269004
36147,-0.350780
...,...
14943,-0.023273
5383,-0.352886
36496,0.340679
52950,0.034033


In [64]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss

#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=1000,
#                                                                     learning_rate=0.15))

#from sklearn.feature_extraction.text import TfidfVectorizer

In [82]:
a = np.arange(0.05, 2.0, 0.02)#wait lol
a
x_train.text

10603    [pm, chest, pa, lat, clip, reason, eval, infil...
39094    [babygram, chest, port, clip, reason, term, ba...
56945    [pm, trauma, port, chest, clip, clip, number, ...
25670    [pm, knee, ap, lat, oblique, right, clip, clip...
6163     [chest, portable, ap, clip, clip, number, radi...
                               ...                        
29765    [video, oropharyngeal, swallow, clip, clip, nu...
345      [pm, mr, head, w, w, contrast, mr, angio, head...
53760    [chest, portable, ap, clip, clip, number, radi...
24961    [knee, ap, lat, oblique, left, clip, clip, num...
12202    [pm, chest, portable, ap, clip, clip, number, ...
Name: text, Length: 38697, dtype: object

In [83]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import make_scorer
auc = make_scorer(roc_auc_score)
pipe1=Pipeline([("wordVectz",AverageEmbeddingVectorizer(WordVectorz)),("multilabel",OneVsRestClassifier(LGBMClassifier(n_estimators = 100, iterations=1000,learning_rate=0.15,n_jobs=-1)))])
#lgbm_model = OneVsRestClassifier(LGBMClassifier(iterations=1500,learning_rate=0.2,n_jobs=-1))
grid_values = {'estimator__learning_rate':a}#1st this alone ok 
grid_clf_acc = OneVsRestClassifier(RandomizedSearchCV(lgbm_model, param_distributions = grid_values,cv=StratifiedKFold(5),scoring = 'roc_auc',n_jobs=-1))#the roc_auc may not work y, i worked with this before i remember it didnt work for me 
grid_clf_acc.fit(x_train.text, y_train)
#hmm ig thats y i didnt prefer pipelining...hey no dude we're messing up something..gridsearch is supposed to improve atleast marginally..
#not really thats what i was going through, there have been many cases where its worse than the base model
# oh wow! thats y i switched to randomsearch hmm can i just run the base one once, a bit scared if thats there..lol go on

ValueError: setting an array element with a sequence.

In [32]:
#print(grid_clf_acc.best_params_)
#print(grid_clf_acc.best_score_)
grid_clf_acc

OneVsRestClassifier(estimator=GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                                           error_score='raise-deprecating',
                                           estimator=OneVsRestClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                                  class_weight=None,
                                                                                                  colsample_bytree=1.0,
                                                                                                  importance_type='split',
                                                                                                  learning_rate=0.2,
                                                                                                  max_depth=-1,
                                                                                                  min_chil

In [72]:
y_pred_acc = grid_clf_acc.predict(x_test)
#y_pred_acc_test = grid_clf_acc.predict(x_test)

In [73]:
print(jaccard_score(y_test,y_pred_acc,average='micro'))
print(jaccard_score(y_test,y_pred_acc,average='macro'))
print(roc_auc_score(y_test,y_pred_acc))
print(hamming_loss(y_test,y_pred_acc))
'''
Cat w2v 500,10,5
0.6633380308436122
0.18253038512227265'''

0.30940802957842706
0.1341012150675117
0.5134799809771451
0.2516327427148924


'\nCat w2v 500,10,5\n0.6633380308436122\n0.18253038512227265'

In [41]:
# on val set
print(jaccard_score(y_val,y_pred_acc,average='micro'))
print(jaccard_score(y_val,y_pred_acc,average='macro'))
print(roc_auc_score(y_val,y_pred_acc))
print(hamming_loss(y_val,y_pred_acc))
'''
Cat w2v 500,10,5
0.6633380308436122
0.18253038512227265'''

0.31409452701274915
0.13868131528419325
0.512668867337527
0.25162273901808785


'\nCat w2v 500,10,5\n0.6633380308436122\n0.18253038512227265'

In [22]:
# tuned lgbm for text model
from sklearn.externals import joblib
joblib.dump(grid_clf_acc, "tuned_lgbm_text.pkl")

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['tuned_lgbm_text.pkl']

predicted1.shape #earlier with 1500d & mincount=3 - 0.6573500885474709

#pipe1.fit(x_train.text,y_train)
predicted1_prob_testdf = pd.DataFrame(predicted1_prob_test)
predicted1_prob_traindf = pd.DataFrame(predicted1_prob_train)

predicted1_prob_test.shape

predicted1_prob_testdf

x_train.text.to_pickle('./train-nlp.pkl')
x_test.text.to_pickle('./test-nlp.pkl')
y_train.to_pickle('./y_train-nlp.pkl')
y_test.to_pickle('./y_test-nlp.pkl')
predicted1_prob_testdf.to_pickle('./pred-test-xgb.pkl')
predicted1_prob_traindf.to_pickle('./pred-train-xgb.pkl')

from sklearn.externals import joblib
joblib.dump(pipe1, "w2v_lgbm.pkl")

textlgbm =  joblib.load("w2v_lgbm.pkl")


In [29]:
#valc = joblib.load("cat1-C.pkl")
#textc = joblib.load("./w2v.pkl")
lgbm_str = joblib.load("./LGBM1-C.pkl")
lgbm_tuned_text = joblib.load("./tuned_lgbm_text.pkl")

In [24]:
x_full = combined_dataset.iloc[:,:484]
y_full = combined_dataset.iloc[:,484:]

In [38]:
x_full.text = x
x_full

,text,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,...,51512,51513,51514,51515,51516,51517,51518,51519,51529,51532
47366,0.242898,7,0,35,462.0,0.0,24.0,26.0,1.0,104.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
47024,0.384075,5,1,60,462.0,-6.0,24.0,17.0,1.0,104.0,...,1.0,1.044,2.0,1.0,7.0,3.0,0.0,0.0,135.5,43.0
9046,-0.154401,13,0,49,462.0,0.0,24.0,26.0,1.0,104.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
843,-0.088762,5,1,61,462.0,2.0,24.0,28.0,1.0,103.0,...,1.0,1.044,1.0,1.0,1.0,3.0,0.0,0.0,135.5,43.0
43466,0.206081,5,0,54,462.0,0.0,23.0,25.0,1.0,106.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24035,-0.167666,25,1,60,653.0,1.0,26.0,28.0,1.3,108.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
30073,-0.073071,10,0,58,343.0,-3.0,24.0,28.0,1.0,104.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
25219,-0.077020,17,1,23,240.0,4.0,24.0,29.0,1.0,104.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
33394,-0.185284,6,1,70,462.0,2.0,24.0,26.0,1.0,105.0,...,1.0,1.044,4.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0


In [39]:
from sklearn.model_selection import train_test_split
seed=243 
x_train_full,x_test_full,y_train_full,y_test_full = train_test_split(x_full,y_full,test_size=0.15,random_state=seed)
x_test_full.iloc[:,1:484]

,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,50808,...,51512,51513,51514,51515,51516,51517,51518,51519,51529,51532
21738,10,1,83,462.0,5.0,24.0,32.0,1.0,104.0,1.12,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
47854,3,1,66,462.0,-2.0,22.0,22.0,1.0,105.0,1.10,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
3615,8,1,80,462.0,1.0,24.0,26.0,1.0,104.0,1.15,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
37273,5,1,46,462.0,0.0,24.0,26.0,1.0,104.0,1.12,...,1.0,1.044,4.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
12132,27,1,85,338.0,-1.0,24.0,25.0,1.0,100.0,1.14,...,1.0,1.044,0.2,1.0,0.0,3.0,0.0,0.0,135.5,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5542,16,1,82,584.0,0.0,24.0,26.0,1.0,104.0,1.12,...,1.0,1.044,0.2,1.0,7.0,3.0,0.0,0.0,135.5,43.0
57534,13,1,56,462.0,0.0,24.0,29.0,1.0,102.0,1.25,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
40358,12,1,36,462.0,0.0,24.0,26.0,1.0,104.0,1.12,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
24988,14,1,50,391.0,6.0,24.0,32.0,1.0,101.0,1.18,...,1.0,1.044,0.2,1.0,0.0,3.0,0.0,0.0,135.5,43.0


In [259]:
y_pred_catnum = valc.predict(x_train_full.iloc[:,1:484])
y_pred_catext = textc.predict(x_train_full.text)

print(y_pred_catnum.shape,y_pred_catext.shape)

(38697, 20) (38697, 20)


In [40]:
# 12/1 9:01
y_pred_lg = lgbm_str.predict(x_train_full.iloc[:,1:484])
y_pred_lg_prob = lgbm_str.predict_proba(x_train_full.iloc[:,1:484])
print(y_pred_lg.shape)

(38697, 20)


In [278]:
y_pred_lgbm = textlgbm.predict(x_test_full.text)#need str stuff ? this is text only, we were using lgbm for str and cat for text
y_pred_lgbm_prob = textlgbm.predict_proba(x_test_full.text)

print(y_pred_lgbm_text.shape)

TypeError: 'int' object is not iterable

In [47]:
#getting proba predictions of tuned lgb on text 12/1 9:02
#lgb_text_proba = lgbm_tuned_text.predict_proba(x_test)
x_test.shape

(6829, 750)

In [203]:
y_pred_lgbm_text = textlgbm.predict(x_test_full.text)
y_pred_lgbm_text_prob = textlgbm.predict_proba(x_test_full.text)

print(y_pred_lgbm_text.shape)

(6829, 20)


In [120]:
y_pred_catnum_prob = valc.predict_proba(x_test_full.iloc[:,1:484])
y_pred_catext_prob = textc.predict_proba(x_test_full.text)
print(y_pred_catnum.shape,y_pred_catext.shape)

(6829, 20) (6829, 20)


In [280]:
#what all is needed lol 
# i need lgbm for str and cat for text on train or test or both? both na?
y_pred_lg_prob_train = lgbm.predict_proba(x_train_full.iloc[:,1:484])
y_pred_lg_prob_test = lgbm.predict_proba(x_test_full.iloc[:,1:484])
y_pred_catext_prob_train = textc.predict_proba(x_train_full.text)
y_pred_catext_prob_test = textc.predict_proba(x_test_full.text)

In [292]:
y_comb =y_pred_lg_prob+
#y_comb/=2
y_comb = y_comb.round()
y_comb = np.where(y_comb>1,1,0)#wait u got this wrong
#just try this, according to me text can give more info, and i would change w2v to 1000 
# 0.7052650131769738
# 0.1599868209108215

In [293]:
np.unique(y_comb)

array([0, 1])

In [294]:
# predicting on avg of both probs on test set
print(jaccard_score(y_test,y_comb,average='micro'))
print(jaccard_score(y_test,y_comb,average='macro'))
print(roc_auc_score(y_test,y_comb))
print(hamming_loss(y_test,y_comb))
#0.6646507472338611

0.07168119091098218
0.13843890622090185
0.561602477838084
0.29404012300483234


In [276]:
# predicting on train set
print(jaccard_score(y_train_full,y_comb,average='micro'))
print(jaccard_score(y_train_full,y_comb,average='macro'))
print(roc_auc_score(y_train_full,y_comb))
print(hamming_loss(y_train_full,y_comb))
#0.6646507472338611

0.6937055799183427
0.648084064314032
0.8135841276855682
0.10468640979920923


In [282]:
auc_sum = 0.7664439688584423/0.17394371656717575 + 0.8135841276855682/0.10468640979920923
#auc/ham worked...
y_comb = (y_pred_lg_prob_test*0.7664439688584423/0.17394371656717575 +y_pred_catext_prob_test*0.8135841276855682/0.10468640979920923)/auc_sum
y_comb = y_comb.round()

In [222]:
import numpy as np
def corr(a, b):
    counts = 0
  #first_df = pd.read_csv(first_file,index_col=0)
  #second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
    for i in range(a.shape[0]):  
      #prediction = a[:,i] #first_df.columns[0]
      # correlation
        #print("Finding correlation row : {}".format(i))
      #print("Column to be measured: {}".format(prediction))
        cor = np.corrcoef(a[i,:],b[i,:])
        #print("Pearson's correlation score: {}".format(cor))
        if(cor[0][1] < 0.7):
            counts+=1
      #print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
      #print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))
    return counts

In [223]:
l = [corr(y_pred_catnum,y_pred_lg)]
l# this was high thats y its working better ig
#what was high? like correlation was low for many inspite o
# 4717 cat v cat
# 4909 lgbm vs cat

[521]

In [ ]:
estimators=[ ('text', textc), ('val', valc)]
ensemble = OneVsRestClassifier(VotingClassifier(estimators, voting='soft'))
ensemble.fit(x_train_mm, y_train)

In [ ]:
def softmax(x):
    return 
    